In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# Come from 
# https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources//Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip
data = pd.read_csv("data/CCLE_expression.csv",index_col=0)

In [3]:
data.shape

(1389, 19177)

In [4]:
data

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,ARHGAP11B (89839),AC004593.2 (1124),AC090517.4 (54816),AL160269.1 (11046),ABCF2-H2BE1 (114483834),POLR2J3 (548644),H2BE1 (114483833),AL445238.1 (647264),GET1-SH3BGR (106865373),AC113348.1 (102724657)
ACH-001113,4.990501,0.000000,7.273702,2.765535,4.480265,0.028569,1.269033,3.058316,6.483171,5.053980,...,1.214125,0.000000,0.111031,0.150560,1.427606,5.781884,0.000000,0.000000,0.799087,0.000000
ACH-001289,5.209843,0.545968,7.070604,2.538538,3.510962,0.000000,0.176323,3.836934,4.200850,3.832890,...,1.835924,0.000000,0.310340,0.000000,0.807355,4.704319,0.000000,0.000000,0.464668,0.070389
ACH-001339,3.779260,0.000000,7.346425,2.339137,4.254745,0.056584,1.339137,6.724241,3.671293,3.775051,...,1.823749,0.084064,0.176323,0.042644,1.384050,4.931683,0.000000,0.028569,0.263034,0.000000
ACH-001538,5.726831,0.000000,7.086189,2.543496,3.102658,0.000000,5.914565,6.099716,4.475733,4.294253,...,0.871844,0.137504,0.263034,2.485427,0.713696,3.858976,0.000000,0.000000,0.000000,0.000000
ACH-000242,7.465648,0.000000,6.435462,2.414136,3.864929,0.831877,7.198003,5.452530,7.112492,4.710944,...,2.324811,0.163499,0.163499,0.000000,1.117695,4.990501,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000750,4.173127,0.000000,6.400879,1.807355,3.303050,0.014355,0.137504,4.944858,4.528571,4.383359,...,1.280956,0.176323,0.084064,0.000000,2.615887,5.102658,0.000000,0.097611,0.214125,0.000000
ACH-000285,0.097611,0.000000,6.428276,3.257011,4.980482,0.411426,0.124328,0.704872,4.829342,5.393348,...,2.939227,0.000000,0.014355,0.000000,3.326250,6.341630,0.000000,0.000000,0.310340,0.000000
ACH-001858,5.045268,0.000000,6.991749,1.807355,3.270529,0.028569,3.333424,6.819796,5.395063,3.727920,...,1.510962,0.056584,0.097611,0.286881,1.925999,4.607626,0.111031,0.000000,1.090853,0.000000
ACH-001997,5.805292,0.000000,7.792855,2.482848,3.903038,0.028569,4.124328,6.816600,4.458119,3.664483,...,0.757023,0.150560,0.432959,0.111031,0.613532,4.787119,0.000000,0.163499,0.084064,0.000000


In [5]:
meta = pd.read_csv("data/sample_info.csv",index_col=0)

In [6]:
meta_cosmic = meta.loc[:,"COSMICID"].dropna(inplace=False).astype({'COSMICID': 'int32'})

In [7]:
meta_cosmic

DepMap_ID
ACH-000001     905933
ACH-000002     905938
ACH-000004     907053
ACH-000006     908148
ACH-000007     907795
               ...   
ACH-002313    1331045
ACH-002314     909773
ACH-002316    1331049
ACH-002317     946358
ACH-002397    2054094
Name: COSMICID, Length: 981, dtype: int32

In [8]:
meta

,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,Achilles_n_replicates,cell_line_NNMD,culture_type,...,primary_or_metastasis,primary_disease,Subtype,age,Sanger_Model_ID,depmap_public_comments,lineage,lineage_subtype,lineage_sub_subtype,lineage_molecular_subtype
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000001,NIH:OVCAR-3,NIHOVCAR3,NIHOVCAR3_OVARY,OVCAR3,905933.0,Female,ATCC,NaN,NaN,NaN,...,Metastasis,Ovarian Cancer,"Adenocarcinoma, high grade serous",60.0,SIDM00105,NaN,ovary,ovary_adenocarcinoma,high_grade_serous,NaN
ACH-000002,HL-60,HL60,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,905938.0,Female,ATCC,NaN,NaN,NaN,...,Primary,Leukemia,"Acute Myelogenous Leukemia (AML), M3 (Promyelo...",35.0,SIDM00829,NaN,blood,AML,M3,NaN
ACH-000003,CACO2,CACO2,CACO2_LARGE_INTESTINE,"CACO2, CaCo-2",NaN,Male,ATCC,NaN,NaN,NaN,...,NaN,Colon/Colorectal Cancer,Adenocarcinoma,NaN,SIDM00891,NaN,colorectal,colorectal_adenocarcinoma,NaN,NaN
ACH-000004,HEL,HEL,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,907053.0,Male,DSMZ,2.0,-3.092311,Suspension,...,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00594,NaN,blood,AML,M6,NaN
ACH-000005,HEL 92.1.7,HEL9217,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,ATCC,2.0,-2.412923,Suspension,...,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00593,NaN,blood,AML,M6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002510,M040416,M040416,M040416_SKIN,NaN,NaN,Male,Academic lab,2.0,-4.241034,Adherent,...,Metastasis,Skin Cancer,"Melanoma, acral",54.0,NaN,NaN,skin,melanoma,acral,NaN
ACH-002511,M140325,M140325,M140325_SKIN,NaN,NaN,Female,Academic lab,2.0,-3.642719,Adherent,...,Primary,Skin Cancer,"Melanoma, mucosal",64.0,NaN,NaN,skin,melanoma,mucosal,NaN
ACH-002512,MM160113,MM160113,MM160113_SKIN,M160113,NaN,Male,Academic lab,2.0,-1.598620,Adherent,...,Metastasis,Skin Cancer,"Melanoma, acral",86.0,NaN,NaN,skin,melanoma,acral,NaN


In [9]:
dose = pd.read_csv("data/secondary-screen-dose-response-curve-parameters.csv",index_col=0)

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
dose.to_csv("data/label_crosstab.csv")

In [11]:
dose_tab = pd.crosstab(index=dose['depmap_id'], columns = dose['name'],values=dose['auc'],aggfunc='mean')

In [12]:
dose_tab

name,1-azakenpaullone,1-naphthyl-PP1,1-phenylbiguanide,10-deacetylbaccatin,10-hydroxycamptothecin,12-O-tetradecanoylphorbol-13-acetate,2'-MeCCPA,"2,3-DCPE",2-methoxyestradiol,3'-fluorobenzylspiperone,...,warfarin,xanomeline,xilobam,xylazine,zaldaride,zaleplon,zardaverine,ziprasidone,zolantidine,zolmitriptan
depmap_id,,,,,,,,,,,,,,,,,,,,,
ACH-000007,1.200721,0.906230,0.840409,0.861057,0.236149,0.678549,NaN,0.977291,0.886259,0.945707,...,NaN,0.849267,0.796757,1.475364,0.880131,NaN,0.832131,0.942748,NaN,1.380169
ACH-000008,0.895310,0.864177,0.986722,0.860633,0.347057,1.042324,1.264433,1.103182,0.896848,1.212847,...,1.166609,0.842375,0.828385,1.134751,0.929585,1.140489,0.888494,0.945813,NaN,0.931826
ACH-000011,0.821426,0.812357,0.894664,0.928655,0.455415,0.551885,1.377388,0.961209,0.907681,0.989196,...,1.211876,0.861407,0.839902,1.358932,0.941054,NaN,0.834375,0.955548,NaN,NaN
ACH-000012,0.874960,0.814528,0.946367,0.816810,0.302450,0.826459,1.381436,0.906660,0.811619,0.894749,...,1.298038,0.837195,0.845846,1.257923,0.916330,0.951544,0.816745,0.923753,0.937082,1.314640
ACH-000013,0.917994,0.860783,0.954582,0.826188,0.087541,0.752544,1.115056,0.842884,0.867457,1.199869,...,1.588719,0.783282,0.815430,1.492371,0.940541,0.893297,0.797681,0.953010,0.960252,0.925308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-001239,NaN,0.883204,0.870239,1.013601,0.318023,NaN,NaN,0.870609,0.791788,0.872277,...,0.856010,0.889074,0.892768,1.160533,0.876945,0.862995,0.999955,0.866009,0.942972,0.829851
ACH-001306,0.894745,0.875309,NaN,1.149388,0.288112,1.000935,1.204651,1.183884,0.875710,1.231729,...,0.969187,0.771182,0.867433,1.116206,0.942135,1.169739,0.718992,1.523193,NaN,1.405732
ACH-001307,1.168534,0.853086,1.193130,0.830150,0.141646,1.199518,1.262466,0.962323,0.805852,1.066176,...,0.935440,0.865896,0.906832,1.079176,0.935193,0.957796,0.787185,1.167778,0.931503,1.062078


In [13]:
intersect_idx = dose_tab.index.intersection(meta_cosmic.index)

In [14]:
ach2cosmic = meta_cosmic[intersect_idx]

In [15]:
ach2cosmic.index

Index(['ACH-000007', 'ACH-000008', 'ACH-000012', 'ACH-000015', 'ACH-000018',
       'ACH-000019', 'ACH-000021', 'ACH-000023', 'ACH-000030', 'ACH-000035',
       ...
       'ACH-000987', 'ACH-000988', 'ACH-000991', 'ACH-000997', 'ACH-001016',
       'ACH-001024', 'ACH-001075', 'ACH-001190', 'ACH-001306', 'ACH-001307'],
      dtype='object', length=348)

In [16]:
drugs = ["I-BET-762","ibrutinib","dabrafenib","trametinib","cisplatin","tamoxifen",#"JQ1"
         "entinostat","gefitinib"]

In [19]:
select_drug_dose = dose_tab.loc[:,drugs]
select_drug_dose = select_drug_dose.reset_index()
select_drug_dose

name,depmap_id,I-BET-762,ibrutinib,dabrafenib,trametinib,cisplatin,tamoxifen,entinostat,gefitinib
0,ACH-000007,0.861505,0.694080,0.936021,0.337852,1.360810,1.686212,0.806977,0.809181
1,ACH-000008,0.942868,1.000002,0.419812,0.578623,1.273482,NaN,0.768721,0.980043
2,ACH-000011,0.860729,0.719978,1.002039,0.897670,0.956842,NaN,0.762133,0.727800
3,ACH-000012,0.818184,0.707159,0.905235,0.903596,0.929012,1.268093,0.743201,0.347400
4,ACH-000013,1.000000,0.831929,0.886404,1.165718,1.007994,NaN,0.821438,0.925410
...,...,...,...,...,...,...,...,...,...
475,ACH-001239,1.000000,1.250051,0.659424,0.520527,0.896832,1.061297,0.735033,1.143546
476,ACH-001306,0.885691,0.978541,0.860659,0.823735,0.918224,1.198565,0.868214,0.942474
477,ACH-001307,1.000000,0.889296,0.692762,0.798080,0.949769,1.242625,0.859609,0.986994
478,ACH-001318,0.907877,0.867007,0.937486,0.843373,0.943757,1.248113,0.708406,0.777800


In [26]:
select_drug_dose = select_drug_dose.set_index("depmap_id")

In [27]:
cosmic_drug_dose = select_drug_dose.loc[ach2cosmic.index,:]
cosmic_drug_dose

name,I-BET-762,ibrutinib,dabrafenib,trametinib,cisplatin,tamoxifen,entinostat,gefitinib
ACH-000007,0.861505,0.694080,0.936021,0.337852,1.360810,1.686212,0.806977,0.809181
ACH-000008,0.942868,1.000002,0.419812,0.578623,1.273482,NaN,0.768721,0.980043
ACH-000012,0.818184,0.707159,0.905235,0.903596,0.929012,1.268093,0.743201,0.347400
ACH-000015,0.584260,0.702234,NaN,0.831460,NaN,1.201367,0.822213,0.920159
ACH-000018,0.890168,0.771700,0.820675,1.075281,0.935819,1.028721,0.955953,1.006019
...,...,...,...,...,...,...,...,...
ACH-001024,0.834191,0.754272,0.891209,0.753278,0.975465,1.164547,0.791574,0.846848
ACH-001075,0.956684,0.749000,1.102488,0.577396,0.975440,1.487849,0.850176,0.694844
ACH-001190,0.797453,1.014290,0.893675,0.621350,0.960127,1.392364,0.715357,0.935362
ACH-001306,0.885691,0.978541,0.860659,0.823735,0.918224,1.198565,0.868214,0.942474


In [28]:
cosmic_drug_dose["COSMIC_ID"] = ach2cosmic.values

In [29]:
ach2cosmic

ACH-000007     907795
ACH-000008     910921
ACH-000012    1240146
ACH-000015     908471
ACH-000018     724812
               ...   
ACH-001024     910853
ACH-001075     753604
ACH-001190     905955
ACH-001306     906795
ACH-001307     924102
Name: COSMICID, Length: 348, dtype: int32

In [30]:
cosmic_drug_dose = cosmic_drug_dose.set_index("COSMIC_ID")
cosmic_drug_dose

name,I-BET-762,ibrutinib,dabrafenib,trametinib,cisplatin,tamoxifen,entinostat,gefitinib
COSMIC_ID,,,,,,,,
907795,0.861505,0.694080,0.936021,0.337852,1.360810,1.686212,0.806977,0.809181
910921,0.942868,1.000002,0.419812,0.578623,1.273482,NaN,0.768721,0.980043
1240146,0.818184,0.707159,0.905235,0.903596,0.929012,1.268093,0.743201,0.347400
908471,0.584260,0.702234,NaN,0.831460,NaN,1.201367,0.822213,0.920159
724812,0.890168,0.771700,0.820675,1.075281,0.935819,1.028721,0.955953,1.006019
...,...,...,...,...,...,...,...,...
910853,0.834191,0.754272,0.891209,0.753278,0.975465,1.164547,0.791574,0.846848
753604,0.956684,0.749000,1.102488,0.577396,0.975440,1.487849,0.850176,0.694844
905955,0.797453,1.014290,0.893675,0.621350,0.960127,1.392364,0.715357,0.935362


In [32]:
# Comes from 
#ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC2_fitted_dose_response_25Feb20.xlsx
# gdsc = pd.read_csv("data/Cell_line_RMA_proc_basalExp.txt",sep="\t")
label = pd.read_excel("data/GDSC2_fitted_dose_response_25Feb20.xlsx")


In [33]:
GDSC_id = list(set(label.COSMIC_ID))

In [34]:
label_info = label.loc[:,["COSMIC_ID","CELL_LINE_NAME","AUC","DRUG_NAME"]]

In [35]:
label_info

,COSMIC_ID,CELL_LINE_NAME,AUC,DRUG_NAME
0,749709,HCC1954,0.975203,Camptothecin
1,749710,HCC1143,0.978464,Camptothecin
2,749711,HCC1187,0.995100,Camptothecin
3,749712,HCC1395,0.867832,Camptothecin
4,749713,HCC1599,0.777532,Camptothecin
...,...,...,...,...
135237,1290798,EFM-192A,0.931696,JQ1
135238,1290905,HCC1428,0.738310,JQ1
135239,1290922,HDQ-P1,0.890749,JQ1
135240,1298157,JIMT-1,0.942634,JQ1


In [38]:
Cap_drugs = ["I-BET-762","Ibrutinib","Dabrafenib","Trametinib","Cisplatin","Tamoxifen",#"JQ1"
         "Entinostat","Gefitinib"]
# selected_drug_sens = label_info.loc[label_info.DRUG_NAME in drugs,:]

In [ ]:
#selected_index = [d in drugs for d in label_info.DRUG_NAME.values]

In [ ]:
#seleced_labels = label_info[selected_index]

In [36]:
selected_alldrug_sens = pd.crosstab(index=label_info['COSMIC_ID'], columns = label_info['DRUG_NAME'],values=label_info['AUC'],aggfunc='mean')

In [39]:
gdsc_tab = selected_alldrug_sens.loc[:,Cap_drugs]
gdsc_tab

DRUG_NAME,I-BET-762,Ibrutinib,Dabrafenib,Trametinib,Cisplatin,Tamoxifen,Entinostat,Gefitinib
COSMIC_ID,,,,,,,,
683665,0.815841,0.986655,0.991812,0.994088,0.964141,0.964728,0.737832,0.984074
683667,0.966871,0.988762,0.987206,0.989145,0.959592,0.912034,0.751731,0.958999
684052,0.937767,0.981702,0.956075,0.965742,0.724311,0.949760,0.883226,0.989079
684057,0.985318,0.958920,NaN,NaN,NaN,NaN,0.790366,NaN
684059,0.987804,0.944549,0.976539,0.989165,0.767399,0.930646,0.852240,0.972875
...,...,...,...,...,...,...,...,...
1660034,0.891130,0.956935,0.965697,0.737116,0.940000,0.922094,0.940733,0.982930
1660035,0.922922,0.982207,NaN,0.733875,0.982650,0.964134,0.746285,0.972723
1660036,0.992947,0.993288,0.994046,0.967038,NaN,NaN,0.975078,NaN


In [40]:
commen_cl_idx = cosmic_drug_dose.index.intersection(gdsc_tab.index)

In [41]:
gdsc_tab_inter = gdsc_tab.loc[commen_cl_idx]
gdsc_tab_inter

DRUG_NAME,I-BET-762,Ibrutinib,Dabrafenib,Trametinib,Cisplatin,Tamoxifen,Entinostat,Gefitinib
COSMIC_ID,,,,,,,,
907795,0.915260,0.970749,0.959899,0.303289,0.968903,NaN,0.784387,NaN
910921,0.870065,0.987993,0.496520,0.610130,0.868631,0.876857,0.783661,0.978473
1240146,0.917116,0.943654,0.973907,0.933458,0.951302,0.918304,0.614504,0.501978
908471,0.930276,0.905646,0.965091,0.885863,0.981518,0.920243,0.928092,0.981741
905946,0.888691,0.979953,0.946242,0.979607,0.963560,0.876912,0.930733,0.979758
...,...,...,...,...,...,...,...,...
906746,0.949196,0.975131,0.983122,0.885407,0.952470,0.955706,0.901627,0.978536
910853,0.937373,0.894458,0.978745,0.943532,0.973326,0.958145,0.913425,0.952790
905955,0.989310,0.993628,0.993756,0.530260,0.995289,0.989109,0.845535,0.990565


In [42]:
ccle_tab_inter = cosmic_drug_dose.loc[commen_cl_idx,drugs]
ccle_tab_inter

name,I-BET-762,ibrutinib,dabrafenib,trametinib,cisplatin,tamoxifen,entinostat,gefitinib
COSMIC_ID,,,,,,,,
907795,0.861505,0.694080,0.936021,0.337852,1.360810,1.686212,0.806977,0.809181
910921,0.942868,1.000002,0.419812,0.578623,1.273482,NaN,0.768721,0.980043
1240146,0.818184,0.707159,0.905235,0.903596,0.929012,1.268093,0.743201,0.347400
908471,0.584260,0.702234,NaN,0.831460,NaN,1.201367,0.822213,0.920159
905946,0.840450,0.927664,0.932337,0.936824,NaN,1.189797,0.862674,0.952597
...,...,...,...,...,...,...,...,...
906746,0.946085,0.872630,1.163850,0.840966,0.942461,0.979148,0.829767,0.899985
910853,0.834191,0.754272,0.891209,0.753278,0.975465,1.164547,0.791574,0.846848
905955,0.797453,1.014290,0.893675,0.621350,0.960127,1.392364,0.715357,0.935362


In [43]:
gdsc_tab_inter.iloc[:,5].name

'Tamoxifen'

In [44]:
corr = []

from scipy import stats
for i in range(0,len(drugs)):
    
    print(gdsc_tab_inter.iloc[:,i].name)
    corr.append(stats.pearsonr(gdsc_tab_inter.iloc[:,i].fillna(1),ccle_tab_inter.iloc[:,i].fillna(1))[0])
    print(stats.pearsonr(gdsc_tab_inter.iloc[:,i].fillna(1),ccle_tab_inter.iloc[:,i].fillna(1)))
    

I-BET-762
(0.07924729592352935, 0.16396908855005227)
Ibrutinib
(0.37462408246414375, 9.153469152781382e-12)
Dabrafenib
(0.5110927519186615, 4.973318666744909e-22)
Trametinib
(0.621079152501503, 1.8547315477786116e-34)
Cisplatin
(0.055491446620167334, 0.3301402914028369)
Tamoxifen
(0.03773559108062024, 0.5079997910161388)
Entinostat
(0.1897922803680579, 0.0007830521381411542)
Gefitinib
(0.4571112345733248, 2.0676967380061648e-17)


In [45]:
corr

[0.07924729592352935,
 0.37462408246414375,
 0.5110927519186615,
 0.621079152501503,
 0.055491446620167334,
 0.03773559108062024,
 0.1897922803680579,
 0.4571112345733248]

In [46]:
import numpy as np
import matplotlib.pyplot as plt
x = np.arange(len(drugs))
plt.bar(x, corr)
plt.xticks(x, drugs,rotation=90)
plt.xlabel('Drugs')
plt.ylabel('Correlation')
plt.title('AUC Correlation')
plt.show()

<Figure size 640x480 with 1 Axes>

In [50]:
GDSC2_df = pd.read_csv("data/GDSC2_expression.csv",index_col=0)

In [51]:
GDSC2_df

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
DATA.683665,3.238273,2.982254,10.235491,4.856061,4.078870,9.116236,3.658590,6.145475,5.042464,5.438402,...,3.103752,3.724013,3.981948,2.823245,5.866047,3.095716,3.274367,3.056214,9.446305,3.530871
DATA.683667,7.780713,2.753253,9.960137,4.351073,3.716740,3.222277,8.221606,3.823474,4.756228,5.805642,...,3.175476,3.779354,4.504481,2.690651,3.347520,3.230713,3.102091,3.169188,9.810430,3.266915
DATA.684052,7.301344,2.890533,9.922489,4.125088,3.678987,3.096576,3.588391,4.809305,4.951782,5.089165,...,3.299300,3.762301,4.177345,2.499803,5.054260,3.003521,3.068187,3.135479,9.073222,3.098364
DATA.684057,8.233101,2.824687,10.015884,4.749715,3.839433,3.142754,5.329830,3.272124,5.538055,6.428482,...,3.081750,3.226083,4.666295,2.491254,6.261573,3.031862,3.322455,2.813440,8.893197,3.266184
DATA.684059,8.333466,3.966757,9.793991,3.976923,3.505669,3.079943,3.373640,4.199048,5.794734,5.902391,...,3.055770,3.738082,4.705392,2.667882,3.885425,2.993918,3.219777,3.033869,8.691401,3.279230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DATA.1659928,7.245503,2.898369,9.737324,4.295495,3.429522,3.264985,3.509126,6.422919,5.522948,4.563243,...,3.302293,3.237528,3.770028,2.650564,5.786714,3.189778,3.528351,3.580661,9.717357,3.704140
DATA.1660034,8.579664,2.778771,9.768011,5.713156,4.455503,3.388605,3.760805,7.483606,6.906877,5.887162,...,3.972611,4.007318,3.605398,2.565349,3.260777,2.983984,3.717966,3.719941,9.798886,3.396734
DATA.1660035,8.077116,2.781325,10.038055,5.205411,3.758069,3.111444,3.714148,8.947314,6.013510,4.406719,...,4.991264,3.312884,4.008429,2.772584,5.787669,3.283547,3.740781,3.471996,8.837456,3.143362
DATA.1660036,7.797699,2.640900,9.463400,4.039697,3.854522,3.212820,3.369225,8.969264,6.660734,4.942390,...,6.654799,3.734381,4.158913,2.849266,4.772469,3.101758,3.662190,3.329809,9.491964,3.317055


In [54]:
GDSC_gname = GDSC2_df.columns.values

In [55]:
GDSC_gname

array(['TSPAN6', 'TNMD', 'DPM1', ..., 'POLRMTP1', 'UBL5P2', 'TBC1D3P5'],
      dtype=object)

In [57]:
import mygene
mg = mygene.MyGeneInfo()



In [62]:
help(mg.query)

Help on method query in module biothings_client.base:

query(q, **kwargs) method of mygene.MyGeneInfo instance
    Return  the query result.
    This is a wrapper for GET query of "/query?q=<query>" service.
    
    :param q: a query string, detailed query syntax `here <http://mygene.info/doc/query_service.html#query-syntax>`_
    :param fields: fields to return, a list or a comma-separated string.
                    If **fields="all"**, all available fields are returned
    :param species: optionally, you can pass comma-separated species names
                    or taxonomy ids. Default: human,mouse,rat.
    :param size:   the maximum number of results to return (with a cap
                   of 1000 at the moment). Default: 10.
    :param skip:   the number of results to skip. Default: 0.
    :param sort:   Prefix with "-" for descending order, otherwise in ascending order.
                   Default: sort by matching scores in decending order.
    :param entrezonly: if True, retu

In [81]:
entrez_ids = []
for gn in GDSC_gname:
    q = mg.query(gn,species='human')
    if(len(q)>0):
        # Select the entrez id with the top score
        try:
            entrez_ids.append(q["hits"][0]['entrezgene'])
        except:
            print("error finding :"+gn)
            entrez_ids.append("unknown")
    else:
        entrez_ids.append("unknown")

error finding :WISP2
error finding :FER1L4
error finding :LRP5L
error finding :KIR3DX1
error finding :SIGLEC5
error finding :KIAA1045
error finding :HMGB1P1
error finding :PPP4R1L
error finding :C22orf24
error finding :LINC00470
error finding :SLC26A10
error finding :ERVK3-1
error finding :CCDC39
error finding :AKR1C6P
error finding :C8orf37
error finding :MSLNL
error finding :LSP1P3
error finding :FAM160A1
error finding :CMAHP
error finding :BRD7P3
error finding :CELP
error finding :RAB4B-EGLN2
error finding :C11orf45
error finding :GRIK1-AS1
error finding :SCAND2P
error finding :LINC00303
error finding :TOB2P1
error finding :C9orf62
error finding :C9orf106
error finding :LINC00311
error finding :HMGN2P46
error finding :OR6W1P
error finding :LINC00652
error finding :CRIPAK
error finding :C9orf139
error finding :EXOC5P1
error finding :OFCC1
error finding :HSP90AA6P
error finding :TP53TG1
error finding :C17orf77
error finding :WT1-AS
error finding :GPR1
error finding :LINC00518
error fi

In [75]:
mg.query("cdk2",species='human')["hits"][0]['entrezgene']

'1017'

In [79]:
entrez_ids

['64102', '8813']

In [47]:
#selected_alldrug_sens.to_csv("data/GDSC2_label_9drugs.csv")

In [ ]:
#select_cells.to_csv("data/GDSC2_expression.csv")